In [ ]:
from google.colab import drive
drive.mount('/content/drive', force_remount = True)

Mounted at /content/drive


In [ ]:
import os
import numpy as np
from PIL import Image
import cv2
import matplotlib.pyplot as plt
from glob import glob
import imageio
import tensorflow as tf
from matplotlib import pyplot
import random
import time
import math

from numpy import load
from numpy import zeros
from numpy import ones
from numpy import savez_compressed
from numpy.random import randint
from keras import backend as K
from keras.optimizers import Adam
from keras.initializers import RandomNormal
from keras.models import Model
from keras.models import Input
from keras.layers import Conv2D
from keras.layers import Conv2DTranspose
from keras.layers import LeakyReLU
from keras.layers import Activation
from keras.layers import Concatenate
from keras.layers import Dropout
from keras.layers import BatchNormalization
from keras.layers import LeakyReLU
from matplotlib import pyplot

In [ ]:
### Hyperparameters ###

#discriminator
lr_dis = 0.0002 
beta_1_dis = 0.5
epsilon_dis = 1e-7
loss_dis = 'binary_crossentropy'
loss_weights_dis = [0.5]

lrsched_dis = False                  #True --> gebruik lr scheduler, False --> niet
lr_change_dis = 2                   #na hoeveel epochs wordt de lr aangepast
drop_dis = 0.5                      #hoeveel neemt de lr af

#gan
lr_gan = 0.0002 
beta_1_gan = 0.5
epsilon_gan = 1e-7
loss_gan = ['binary_crossentropy', 'mse']
loss_weights_gan = [1,100]

lrsched_gan = False                  #True --> gebruik lr scheduler, False --> niet
lr_change_gan = 3                   #na hoeveel epochs wordt de lr aangepast
drop_gan = 0.5                      #hoeveel neemt de lr af

#train loop
nr_epochs = 50
batch_size= 4
n_epochs_stop = 5

# dropout in decoder generator


#
main_data_path = '/content/drive/Shareddrives/AI for Health/Data/'
npz_dir = main_data_path+'train/'

#CHANGE TO YOUR NAME
model_name = 'baseline_sigm.h5'
save_path = '/content/drive/Shareddrives/AI for Health/Models/Myrthe vd Berg/'+model_name[:-3]+'/'
if os.path.exists(save_path) == False:
  os.mkdir(save_path)


**Data Inladen**

Real samples validatie & train
data inladen
shapen naar [-1,1]
reshapen met extra grayscale dimensie
train/val split

In [ ]:
def load_real_samples(filename):
    # load compressed arrays
    data = load(filename)

    # unpack arrays
    X1, X2 = data['arr_0'], data['arr_1']

    # scale from [0,255] to [0,1]
    X1 = X1/255
    X2 = X2/255
    # reshape data (add 3rd grayscale dimension for 2d convolution layers)
    X1 = X1.reshape(len(X1), 512, 512, 1)
    X2 = X2.reshape(len(X1), 512, 512, 1)

    #noise = np.random.normal(0, .1, X1.shape)
    #X1 = X1 + noise
    ind = list(range(0,len(X1)))
    random.seed(1)
    random.shuffle(ind)

    train = [X1[ind[:792]], X2[ind[:792]]]
    val = [X1[ind[792:]], X2[ind[792:]]]
    return train, val, ind 

train_dataset, val_dataset, ind = load_real_samples(npz_dir+'train_dataset.npz')

directory_train_files = os.listdir(main_data_path+'train/train_images/') 
trn_img_names = [file for file in directory_train_files if file.endswith('.png') or file.endswith('.jpg')]
trn_img_names.sort()
train_names = [trn_img_names[i] for i in ind[:792]]
val_names = [trn_img_names[i] for i in ind[792:]]

print(train_dataset[0].shape, train_dataset[1].shape, val_dataset[0].shape, val_dataset[1].shape)
img_shape = train_dataset[0].shape[1:]

(792, 512, 512, 1) (792, 512, 512, 1) (98, 512, 512, 1) (98, 512, 512, 1)


**Discriminator**

Functie voor discriminator

Initialisatie discriminator

In [ ]:
# define the discriminator model
def define_discriminator(image_shape=(512,512), lr=0.0002, beta_1=0.5, epsilon=1e-7, loss='binary_crossentropy', loss_weights=[0.5]):
	# weight initialization
	init = RandomNormal(stddev=0.02)
	# source image input
	in_src_image = Input(shape=image_shape)
	# target image input
	in_target_image = Input(shape=image_shape)
	# concatenate images channel-wise
	merged = Concatenate()([in_src_image, in_target_image])
	# C64
	d = Conv2D(64, (4,4), strides=(2,2), padding='same', kernel_initializer=init)(merged)
	d = LeakyReLU(alpha=0.2)(d)
	# C128
	d = Conv2D(128, (4,4), strides=(2,2), padding='same', kernel_initializer=init)(d)
	d = BatchNormalization()(d)
	d = LeakyReLU(alpha=0.2)(d)
	# C256
	d = Conv2D(256, (4,4), strides=(2,2), padding='same', kernel_initializer=init)(d)
	d = BatchNormalization()(d)
	d = LeakyReLU(alpha=0.2)(d)
	# C512
	d = Conv2D(512, (4,4), strides=(2,2), padding='same', kernel_initializer=init)(d)
	d = BatchNormalization()(d)
	d = LeakyReLU(alpha=0.2)(d)
	# second last output layer
	d = Conv2D(512, (4,4), padding='same', kernel_initializer=init)(d)
	d = BatchNormalization()(d)
	d = LeakyReLU(alpha=0.2)(d)
	# patch output
	d = Conv2D(1, (4,4), padding='same', kernel_initializer=init)(d)
	patch_out = Activation('sigmoid')(d)
	# define model
	model = Model([in_src_image, in_target_image], patch_out)
	# compile model
	opt = Adam(lr=lr, beta_1=beta_1, epsilon=epsilon)
	model.compile(loss=loss, optimizer=opt, loss_weights=loss_weights)
	return model

d_model = define_discriminator(img_shape, lr_dis, beta_1_dis, epsilon_dis, loss_dis, loss_weights_dis)

**Generator**

Code

Initialisatie

In [ ]:
# define an encoder block
def define_encoder_block(layer_in, n_filters, batchnorm=True):
	# weight initialization
	init = RandomNormal(stddev=0.02)
	# add downsampling layer
	g = Conv2D(n_filters, (4,4), strides=(2,2), padding='same', kernel_initializer=init)(layer_in)
	# conditionally add batch normalization
	if batchnorm:
		g = BatchNormalization()(g, training=True)
	# leaky relu activation
	g = LeakyReLU(alpha=0.2)(g)
	return g
 
# define a decoder block
def decoder_block(layer_in, skip_in, n_filters, dropout=False):
	# weight initialization
	init = RandomNormal(stddev=0.02)
	# add upsampling layer
	g = Conv2DTranspose(n_filters, (4,4), strides=(2,2), padding='same', kernel_initializer=init)(layer_in)
	# add batch normalization
	g = BatchNormalization()(g, training=True)
	# conditionally add dropout
	if dropout:
		g = Dropout(0.5)(g, training=True)
	# merge with skip connection
	g = Concatenate()([g, skip_in])
	# leaky relu activation
	g = LeakyReLU(alpha=0.2)(g)
	return g
 
# define the standalone generator model
def define_generator(image_shape=(512,512)):
    # weight initialization
    init = RandomNormal(stddev=0.02)
    # image input
    in_image = Input(shape=image_shape)
    # encoder model
    e1 = define_encoder_block(in_image, 64, batchnorm=False)
    e2 = define_encoder_block(e1, 128)
    e3 = define_encoder_block(e2, 256)
    e4 = define_encoder_block(e3, 512)
    e5 = define_encoder_block(e4, 512)
    e6 = define_encoder_block(e5, 512)
    e7 = define_encoder_block(e6, 512)
    # bottleneck, no batch norm and relu
    b = Conv2D(512, (4,4), strides=(2,2), padding='same', kernel_initializer=init)(e7)
    b = LeakyReLU(alpha=0.2)(b)
    # decoder model
    d1 = decoder_block(b, e7, 512)
    d2 = decoder_block(d1, e6, 512)
    d3 = decoder_block(d2, e5, 512)
    d4 = decoder_block(d3, e4, 512, dropout=True)
    d5 = decoder_block(d4, e3, 256, dropout=True)
    d6 = decoder_block(d5, e2, 128, dropout=True)
    d7 = decoder_block(d6, e1, 64, dropout=False)
    # output
    g = Conv2DTranspose(1, (4,4), strides=(2,2), padding='same', kernel_initializer=init)(d7)
    out_image = Activation('sigmoid')(g)
    # define model
    model = Model(in_image, out_image)
    return model

g_model = define_generator(img_shape)

**Compile GAN**

In [ ]:
# define the combined generator and discriminator model, for updating the generator
def define_gan(g_model, d_model, image_shape=(512,512), lr=0.0002, beta_1=0.5, epsilon=1e-7, loss=['binary_crossentropy', 'mae'], loss_weights=[1,100]):
	# make weights in the discriminator not trainable
	for layer in d_model.layers:
		if not isinstance(layer, BatchNormalization):
			layer.trainable = False
	# define the source image
	in_src = Input(shape=image_shape)
	# connect the source image to the generator input
	gen_out = g_model(in_src)
	# connect the source input and generator output to the discriminator input
	dis_out = d_model([in_src, gen_out])
	# src image as input, generated image and classification output
	model = Model(in_src, [dis_out, gen_out])
	# compile model
	opt = Adam(lr=lr, beta_1=beta_1, epsilon=epsilon)
	model.compile(loss=loss, optimizer=opt, loss_weights=loss_weights)
	return model

gan_model = define_gan(g_model, d_model, img_shape, lr_gan, beta_1_gan, epsilon_gan, loss_gan, loss_weights_gan)

**Fake & real samples**

In [ ]:
# generate a batch of images, returns images and targets
def generate_fake_samples(g_model, samples, patch_shape):
	# generate fake instance
	X = g_model.predict(samples)
	# create 'fake' class labels (0)
	y = zeros((len(X), patch_shape, patch_shape, 1))
	return X, y
def generate_real_samples(dataset, n_samples, patch_shape):
	# unpack dataset
	trainA, trainB = dataset
	# choose random instances
	ix = randint(0, trainA.shape[0], n_samples)
	# retrieve selected images
	X1, X2 = trainA[ix], trainB[ix]
	# generate 'real' class labels (1)
	y = ones((n_samples, patch_shape, patch_shape, 1))
	return [X1, X2], y 

**Performance functies**

functie maken voor:

--> loss functie validatie + opslaan in lijst voor plot

--> opslaan van generator model als validatie loss lager is

--> early stopping (als validatie loss niet meer afneemt)

ZS dorian:
* Disc loss
* Gen loss
* 3 plots of example images with dice scores 
* Save of the model
* The amount of epochs




In [ ]:
def dice(pred, true, k = 1):
    intersection = np.sum(pred[true==k]) * 2.0
    dice = intersection / (np.sum(pred) + np.sum(true))
    return dice


In [ ]:
def summarize_val_performance(epoch, g_model, val_dataset, n_epochs_stop, model_name, save_path, mean_val_loss, std_val_loss):
    #validatie loss bepalen
    #Opslaan g_model --> model_name uit hyperparameter selectie
    #Early stopping checken

    val_losses = list()
    best_loss = list()

    #for plotting
    #save_plot_ind = randint(0, len(val_dataset[0]), 3)
    row = 0 
    array = np.zeros((len(val_dataset[0]),512, 512,1))
    ### Predict val set + Loss ###
    for idx in range(len(val_dataset[0])):
        loss = list()
        src_image = val_dataset[0][idx]
        src_image = np.expand_dims(src_image, 0)
        gen_image = g_model.predict(src_image)
        gt_image = val_dataset[1][idx]
        # scale from [-1,1] to [0,1]
        #src_image = (src_image + 1) / 2.0
        gen_image = np.where(gen_image > 0.5, 1, 0)
        #gt_image = (gt_image + 1) / 2.0

        ### CALCULATE LOSS ###
        loss = -dice(gen_image.reshape(512,512), gt_image.reshape(512,512))#np.mean(np.square(gt_image-gen_image))#
        val_losses.append(loss)

        #gen_image = np.where(gen_image > 0.5, 1, 0)
        array[idx]=gen_image
        ### CREATE FIGURE ###
    val_array = np.array(val_losses)
    plotlist = val_array.argsort()[-2:][::-1]
    plotlist = list(plotlist)
    plotlist.append(val_losses.index(np.min(val_losses)))
    #if idx in save_plot_ind:
    for idx in plotlist:
      if row == 0:
        fig, ax = plt.subplots(3,3, figsize=(17,17))
        fig.subplots_adjust(hspace=0.5)

      ax[row,0].imshow(val_dataset[0][idx].reshape(512,512), cmap='gray')
      ax[row,0].axis('off')
      ax[row,0].set_title(val_names[idx])

      ax[row,1].imshow(array[idx].reshape(512,512), cmap='gray')
      ax[row,1].axis('off')
      ax[row,1].set_title('dice={:.3f}'.format(-val_losses[idx]))

      ax[row,2].imshow(val_dataset[1][idx].reshape(512,512), cmap='gray')
      ax[row,2].axis('off')
      ax[row,2].set_title('True mask')
      
      row +=1
    
    ### SAVE FIGURE ###
    filename1 = save_path+'plot_{}_ep_{}.png'.format(model_name, epoch)
    pyplot.savefig(filename1)
    plt.close()

    mean_val_loss.append(np.mean(val_losses))
    std_val_loss.append(np.std(val_losses))
    return mean_val_loss, std_val_loss

def summarize_train_performance(epoch, g_model, train_dataset, mean_train_loss, std_train_loss):
    #validatie loss bepalen
    #Opslaan g_model --> model_name uit hyperparameter selectie
    #Early stopping checken

    train_losses = list()

    ### Predict train set + Loss ###
    for idx in range(len(train_dataset[0])):
        loss = list()
        src_image = train_dataset[0][idx]
        src_image = np.expand_dims(src_image, 0)
        gen_image = g_model.predict(src_image)
        gt_image = train_dataset[1][idx]
        # scale from [-1,1] to [0,1]
        #src_image = (src_image + 1) / 2.0
        gen_image = np.where(gen_image > 0.5, 1, 0)
        #gt_image = (gt_image + 1) / 2.0

        ### CALCULATE LOSS ###
        loss = -dice(gen_image.reshape(512,512), gt_image.reshape(512,512))#np.mean(np.square(gt_image-gen_image))#
        train_losses.append(loss)
    

    mean_train_loss.append(np.mean(train_losses))
    std_train_loss.append(np.std(train_losses))

    return mean_train_loss, std_train_loss




**Training loop**

idee:

```
for epoch in range(epochs)
    for step in range(steps)
        train discriminator met real samples
        train discriminator met fake samples
        update generator met 'define_gan' functie

        --> d1_loss, d2_loss & g_loss opslaan
    
    validatie loss bepalen
    opslaan generator model als loss lager is
    early stopping checken

```




In [ ]:
# train pix2pix models


def train(d_model, g_model, gan_model, dataset, n_epochs, n_batch):

    plot_d_loss1 = list()
    plot_d_loss2 = list()
    plot_g_loss = list()
    mean_val_loss = list()
    mean_train_loss = list()
    std_val_loss = list()
    std_train_loss = list()

    # determine the output square shape of the discriminator
    n_patch = d_model.output_shape[1]
    # unpack dataset
    trainA, trainB = dataset
    # calculate the number of batches per training epoch
    bat_per_epo = int(len(trainA) / n_batch)

    # manually enumerate epochs
    for epoch in range(n_epochs):
        ########set learning rate with lr schedule################
        if lrsched_dis == True and epoch % lr_change_dis == 0:
            new_lr_dis = lr_dis * math.pow(drop_dis, math.floor((1+epoch)/lr_change_dis))
            K.set_value(d_model.optimizer.learning_rate, new_lr_dis)
            print('Lr dis changed to {}'.format(d_model.optimizer.learning_rate))

        if lrsched_gan == True and epoch % lr_change_gan == 0:
            new_lr_gan = lr_gan * math.pow(drop_gan, math.floor((1+epoch)/lr_change_gan))
            K.set_value(gan_model.optimizer.learning_rate, new_lr_gan)
            print('Lr gan changed to {}'.format(gan_model.optimizer.learning_rate))
          
        for i in range(bat_per_epo):
          print("\r Epoch {} progress: {} % ".format(epoch, np.round((i+1)/bat_per_epo*100,2)), end="")
          # select a batch of real samples
          [X_realA, X_realB], y_real = generate_real_samples(dataset, n_batch, n_patch)
          # generate a batch of fake samples
          X_fakeB, y_fake = generate_fake_samples(g_model, X_realA, n_patch)
          # update discriminator for real samples
          d_loss1 = d_model.train_on_batch([X_realA, X_realB], y_real)
          # update discriminator for generated samples
          d_loss2 = d_model.train_on_batch([X_realA, X_fakeB], y_fake)
          # update the generator
          g_loss, _, _ = gan_model.train_on_batch(X_realA, [y_real, X_realB])
          # summarize performance
          
          ########Losses opslaan################
          plot_d_loss1.append(d_loss1) 
          plot_d_loss2.append(d_loss2)
          plot_g_loss.append(g_loss)
          
          #print('>%d, d1[%.3f] d2[%.3f] g[%.3f]' % (i+1, d_loss1, d_loss2, g_loss))
          
          
        # save losses in case model crashes
        npzname = save_path+'loss_list.npz'
        savez_compressed(npzname, plot_g_loss, plot_d_loss1, plot_d_loss2)
        # check validatie loss, model opslaan, early stopping

        mean_val_loss, std_val_loss = summarize_val_performance(epoch, g_model, val_dataset, n_epochs_stop, model_name, save_path, mean_val_loss, std_val_loss)
        mean_train_loss, std_train_loss = summarize_train_performance(epoch, g_model, train_dataset, mean_train_loss, std_train_loss)
        print('epoch: ', epoch, '\ttrain loss: {:.4f} std: {:.4f}'.format(mean_train_loss[-1], std_train_loss[-1]), '\tval loss: {:.4f} std: {:.4f}'.format(mean_val_loss[-1], std_val_loss[-1]))

        ### SAVE BEST MODEL ###
        #Keep track of best generalizing model
        #First run to log model & val_loss
        if epoch == 0:
          best_model = g_model
          best_loss = mean_val_loss[0]
          best_model.save(save_path+model_name)
        #Check if loss is lower than previous epoch, if so save model as best_model & save the val loss
        if epoch > 0 and mean_val_loss[-1] < best_loss:
          best_model = g_model
          best_model.save(save_path+model_name)
          best_loss = mean_val_loss[-1]
          best_epoch = epoch
          print('logged best model, val loss = ', best_loss)

        npzname2 = save_path+'epoch_loss_list.npz'
        savez_compressed(npzname2, mean_val_loss, mean_train_loss, std_val_loss, std_train_loss)

        ### EARLY STOPPING ###
        if epoch == 0:
          epochs_no_improve = 0 
        #for early stopping check if the loss is lower than the previous epoch, if yes, epochs_no_improve =0            
        if epoch > 0 and (mean_val_loss[-1] < mean_val_loss[-2] and ((-mean_val_loss[-1]) - (-mean_val_loss[-2])) > 0.001):
            epochs_no_improve = 0
        #for early stopping check if the loss is lower than the previous epoch, if not, epochs_no_improve +=1            
        if epoch > 0 and (mean_val_loss[-1] > mean_val_loss[-2] or ((-mean_val_loss[-1]) - (-mean_val_loss[-2])) < 0.001):
          epochs_no_improve += 1     
          print('Not improved count: ', epochs_no_improve)
        #if epochs_no_improve is the count where we want the model to stop, break the epoch forloop
        if epoch > 0 and epochs_no_improve == n_epochs_stop:
          print('No more improvement, break')
          break
    return mean_val_loss, std_val_loss, mean_train_loss, std_train_loss, best_loss, best_epoch, plot_g_loss, plot_d_loss1, plot_d_loss2

In [ ]:
with tf.device('/device:GPU:0'):
  start_time = time.time()
  mean_val_loss, std_val_loss, mean_train_loss, std_train_loss, best_val_loss, best_epoch, plot_g_loss, plot_d_loss1, plot_d_loss2 = train(d_model, g_model, gan_model, train_dataset, nr_epochs, batch_size)
  print("--- %s minutes ---" % ((time.time() - start_time)/60))
  print('Epoch: {}, best val loss: {}'.format(best_epoch, best_val_loss))

 Epoch 0 progress: 29.29 % 

In [ ]:
def plot_loss(g_loss, d1_loss, d2_loss):
  # summarize history for loss
  fig, (ax1, ax2, ax3) = plt.subplots(3, 1, figsize=(12,12), sharex=True)
  fig.subplots_adjust(hspace=0.1)

  ax1.plot(g_loss, 'g')
  ax1.set_ylabel('Generator loss')
  ax1.set_xlim(0, len(g_loss)-1)
 

  ax2.plot(d1_loss, 'r')
  ax2.set_ylabel('Real images loss')
  ax2.set_xlim(0, len(d1_loss)-1)


  ax3.plot(d2_loss, 'b')
  ax3.set_ylabel('Fake images loss')
  ax3.set_xlim(0, len(d2_loss)-1)
 
 
  plt.xlabel('steps')
  filename2 = save_path+'losses_{}.png'.format(model_name)
  pyplot.savefig(filename2)
  plt.show()

In [ ]:
def plot_avg_step_loss(g_loss, d1_loss, d2_loss, nr_epochs, batch_size):
  # summarize history for loss
  fig, (ax1, ax2, ax3) = plt.subplots(3, 1, figsize=(12,12), sharex=True)
  fig.subplots_adjust(hspace=0.1)

  steps_per_epoch = int(1576 / batch_size)
  avg_idx = [steps_per_epoch] * nr_epochs

  g_loss_copy = list(g_loss[:])
  g_loss_means = [sum(g_loss_copy.pop(0) for _ in range(i)) / i for i in avg_idx]
  
  d1_loss_copy = list(d1_loss[:])
  d1_loss_means = [sum(d1_loss_copy.pop(0) for _ in range(i)) / i for i in avg_idx]

  d2_loss_copy = list(d2_loss[:])
  d2_loss_means = [sum(d2_loss_copy.pop(0) for _ in range(i)) / i for i in avg_idx]

  ax1.plot(g_loss_means, 'g')
  ax1.set_ylabel('Generator loss')
  ax1.set_xlim(0, len(g_loss_means)-1)
 

  ax2.plot(d1_loss_means, 'r')
  ax2.set_ylabel('Real images loss')
  ax2.set_xlim(0, len(d1_loss_means)-1)


  ax3.plot(d2_loss_means, 'b')
  ax3.set_ylabel('Fake images loss')
  ax3.set_xlim(0, len(d2_loss_means)-1)
 
 
  plt.xlabel('Epoch')
  filename2 = save_path+'mean_losses_{}.png'.format(model_name)
  pyplot.savefig(filename2)
  plt.show()

In [ ]:
data = load(save_path+'loss_list.npz')

# unpack arrays
plot_g_loss, plot_d_loss1, plot_d_loss2 = data['arr_0'], data['arr_1'], data['arr_2']
plot_avg_step_loss(plot_g_loss, plot_d_loss1, plot_d_loss2, 50, batch_size) # set to 14 because I interrupted model training after epoch 14 because of time

In [ ]:
data = load(save_path+'loss_list.npz')

# unpack arrays
plot_g_loss, plot_d_loss1, plot_d_loss2 = data['arr_0'], data['arr_1'], data['arr_2']
plot_loss(plot_g_loss, plot_d_loss1, plot_d_loss2)

In [ ]:
data = load(save_path+'epoch_loss_list.npz')

# unpack arrays
mean_val_loss, mean_train_loss = data['arr_0'], data['arr_1']

plt.figure()
plt.plot(mean_val_loss, label='Validation loss')
plt.plot(mean_train_loss, label='Training loss')
plt.xlabel('Epoch')
plt.ylabel('-dice value')
plt.legend()
filename3 = save_path+'epoch_trainval_loss_{}.png'.format(model_name)
pyplot.savefig(filename3)